In [1]:
#Load data and test data
import os
import json

udemy_data = list()
filenames = os.listdir("udemy_data")
for i, f in enumerate(filenames):
    course_data = json.load(open("udemy_data/" + f))
    udemy_data.append(course_data)
    #print("Done {}/{}".format(i+1, len(filenames)))
print("Done")
print(udemy_data[3000]["t"])

Done
Aprende Programación con Diagramas de Flujo


In [2]:
#Create topics counter and search function
from collections import Counter

topics_counter = Counter()
for course in udemy_data:
    #search_list.add(course["t"].lower())
    for c in course["c"]:
        topics_counter.update([c["t"].lower()])
        #for c2 in c["c"]:
            #search_list.add(c2["t"].lower())          
print(len(topics_counter))

def search_topics(term):
    term = term.lower()
    #Return words that got the term, sorting by most used
    results = [topic for topic in topics_counter.keys() if term in topic]
    
    #return sorted results by count
    return sorted(results, key=lambda a: topics_counter[a], reverse=True)

29986


In [3]:
#Get topics prereqs and function to get prereqs
from collections import Counter, defaultdict

topics_prereqs = defaultdict(Counter)
for course in udemy_data: #Iterate thru every course
    course_contents = list() #List to register the current course contents
    for content in course["c"]: #Iterate thru content
        #For the current content, append every one that happened before it as pre reqs
        topics_prereqs[content["t"].lower()].update(course_contents)
        course_contents.append(content["t"].lower())
        
def get_topic_prereqs(topic):

    #subtract occurrences of the target knowledge from the pre reqs candidates
    #Must take care with low occurrences contents titles because they will tend to stay at 1, since they happen low times and rarely are cited by other things

    prereqs_diff = Counter()
    for prereq in topics_prereqs[topic].keys():

        #Use get function to avoid modify the prereqs counter dict
        prereq_count = topics_prereqs[topic].get(prereq, 0) #Get the degree of need of the prereq in the target
        topic_count = topics_prereqs[prereq].get(topic, 0) #Get the degree of need of the target in the prereq, return 0 if not found

        prereqs_diff[prereq] = prereq_count - topic_count
        
    return prereqs_diff.most_common()



In [89]:
search_topics("c#")

['c# basics',
 'apostila - c# e orientação a objetos',
 'overview of c#',
 'classes in c#',
 'create course project & c# refresher + c# 6.0 new features',
 'learn c# in 1 hour',
 'nail c# in 5 lectures',
 'asp.net ve c# ile fonksiyonları kodluyoruz',
 'oops concepts in c# - classes and objects',
 'c# input and output',
 'additional c# control flow constructs',
 'c# dil özellikleri',
 'python in c# (c# dynamic language support)',
 'c# first project',
 'introducción a la programación mecánicas de videojuegos con c#',
 'conditional statements or selection statements in c#',
 'a starting primer for the c# programming language',
 'new features in c# 6',
 'c# : sınıf ,metod ve özellikler ile çalışmak',
 'newbie: c# primer',
 'features of c# 6.0',
 'the c# language - grasp the basics (optional)',
 'c#.net language basics',
 'tricks and hacks in c# and .net',
 'an introduction to the c# language',
 'loops in c#',
 'introducción a la programación c#',
 'learn c# programming : decisions and loop

In [5]:
topic_prereqs = get_topic_prereqs("loops")
#topic_prereqs = topics_prereqs["javascript"].most_common()
print("\n".join(map(str, topic_prereqs)))

('introduction', 15)
('conditional statements', 10)
('getting started', 8)
('operators', 6)
('variables', 6)
('basics', 4)
('conditionals', 3)
('numbers', 3)
('variables and data types', 3)
('if statements', 2)
('conditions', 2)
('variables and types', 2)
('setup', 2)
('welcome', 2)
('introduction to javascript', 2)
('python programming introduction', 2)
('advanced functions', 2)
('operators in javascript', 2)
('index', 2)
('decision making in python', 2)
('decision making', 2)
('control flow', 1)
('sass overview', 1)
('setting up python', 1)
('if-else conditions', 1)
('media queries', 1)
('conditional statments', 1)
('making decisions  — if-elif-else', 1)
('bonus 1: useful shortcus', 1)
('types, variables, user input and flow control statements', 1)
('basics of javascript', 1)
('numbers and variables in python', 1)
('bringing javascript into action!', 1)
('mixins', 1)
('operators in c++', 1)
('general syntax', 1)
('perl basics', 1)
('more operations', 1)
('primitive data types,operato

In [6]:
#Create function to get the prereqs that occurs above a certain cut value
def get_cutted_prereq(topic, cut_value=0):
    #get prereqs for the topic
    topic_prereqs = get_topic_prereqs(topic)
    cutted_prereqs = [pr for pr, count in topic_prereqs if count > cut_value]
    return cutted_prereqs

#Create function to get the entire prereq flow recursively
def get_prereq_flow_members(topic,cut_value, prereq_dict):
    topic_prereqs = get_cutted_prereq(topic, cut_value)
    prereq_dict[topic] = topic_prereqs
    
    for pr in topic_prereqs:        
        if pr not in prereq_dict:
            prereq_dict = get_prereq_flow_members(pr, cut_value, prereq_dict)
        
    return prereq_dict    

In [58]:
#Create function to create the spanning tree of prereq flows, that is the optimal "learning path"
#With optimal we mean learn only what is necessary and on the time that is necessary
import networkx as nx

def get_topic_prereq_flow_graph(topic, cut_value=0):
    prereq_flow_members = get_prereq_flow_members(topic, cut_value, dict())

    #MUST CHECK IF THE PROCESS TIL HERE ALREADY AVOID CREATION OF CIRCLES
    flow_graph = nx.DiGraph()
    
    for source, targets in prereq_flow_members.items():
        for target in targets:
            flow_graph.add_edge(source, target)
            
    return flow_graph  
    
    
def get_optimal_topic_prereq_flow_graph(topic, cut_value=0):
    #we treat every branch as diferent path to learn something,
    #But if we find some of the branchs inside other branch, we may remove it
    #In order to have a more clean visualization
    
    #The data origin naturally avoid parallel needing (MUST CHECK THIS)
    #MUST FIND A WAY TO DETECT TWO PARALLEL NEEDING AND TWO PATHS TO THE SAME THING
    #this is parallel needing since we take everything before and each thing we add as a need
    
    flow_graph = get_topic_prereq_flow_graph(topic, cut_value)
    optimal_flow_graph = nx.Edmonds(flow_graph)
    return optimal_flow_graph


In [141]:
from tabulate import tabulate

def print_data(headers, data_list):
    """Function to print data tabulated """
    return print(tabulate(data_list, headers=headers))

def print_optimal_topic_prereq_flow(topic, cut_value=0):
    prereq_flow_members = get_prereq_flow_members(topic, cut_value, dict())
    
    #1. Construct graph
    flow_graph = nx.DiGraph()
    for source, targets in prereq_flow_members.items():
        for target in targets:
            flow_graph.add_edge(source, target)
    
    #2. Get node levels
    node_levels = dict()
    
    def get_node_level(node):
        node_level = 0
        for target in flow_graph[node].keys():
            target_level = get_node_level(target)
            node_level = max(target_level+1, node_level)
        return node_level
    
    for n in flow_graph.nodes():
        node_levels[n] = get_node_level(n)
        
    #3. Create print table
    print_tables = defaultdict(list)
    for node, level in node_levels.items():
        print_tables[level].append(node)
        
    #4. Print topics prereq flow
    print(tabulate(print_tables))
        
    return flow_graph
    

In [164]:
flow_graph = print_optimal_topic_prereq_flow("loops", 2)
#print(flow_graph.edges())
#flow_graph = print_optimal_topic_prereq_flow("jquery", 7)
flow_graph.edges()
keep on it!!

--------------------------  ---------------  ---------  ----------------------  -----
introduction to javascript  basics           operators  conditional statements  loops
welcome                     variables
numbers                     getting started
conditionals
introduction to the course
controlling program flow
variables and data types
course introduction
introduction
intro
--------------------------  ---------------  ---------  ----------------------  -----


[('basics', 'introduction'),
 ('conditional statements', 'operators'),
 ('conditional statements', 'introduction'),
 ('conditional statements', 'getting started'),
 ('conditional statements', 'introduction to javascript'),
 ('variables', 'introduction'),
 ('variables', 'course introduction'),
 ('operators', 'controlling program flow'),
 ('operators', 'introduction'),
 ('operators', 'basics'),
 ('operators', 'variables'),
 ('operators', 'introduction to the course'),
 ('loops', 'basics'),
 ('loops', 'conditional statements'),
 ('loops', 'variables'),
 ('loops', 'operators'),
 ('loops', 'introduction'),
 ('loops', 'numbers'),
 ('loops', 'conditionals'),
 ('loops', 'getting started'),
 ('loops', 'variables and data types'),
 ('getting started', 'introduction'),
 ('getting started', 'introduction to the course'),
 ('getting started', 'welcome'),
 ('getting started', 'intro'),
 ('getting started', 'course introduction')]

In [34]:
get_prereq_flow_members("css", 2, dict())

{'course introduction': [],
 'css': ['html', 'introduction', 'getting started'],
 'getting started': ['introduction',
  'course introduction',
  'intro',
  'welcome',
  'introduction to the course'],
 'html': ['introduction', 'preparation', 'getting started'],
 'intro': [],
 'introduction': [],
 'introduction to the course': [],
 'preparation': [],
 'welcome': []}

In [31]:
get_optimal_topic_prereq_flow("css", 2)

[('html', 'introduction'), ('css', 'html'), ('getting started', 'introduction')]
[('html', 'introduction'), ('html', 'preparation'), ('html', 'getting started'), ('css', 'html'), ('css', 'introduction'), ('css', 'getting started'), ('getting started', 'introduction'), ('getting started', 'course introduction'), ('getting started', 'intro'), ('getting started', 'welcome'), ('getting started', 'introduction to the course')]


In [32]:
flow_graph = get_topic_prereq_flow_graph("css", 2)
flow_graph.edges()

[('html', 'introduction'),
 ('html', 'getting started'),
 ('html', 'preparation'),
 ('css', 'html'),
 ('css', 'introduction'),
 ('css', 'getting started'),
 ('getting started', 'introduction'),
 ('getting started', 'introduction to the course'),
 ('getting started', 'welcome'),
 ('getting started', 'intro'),
 ('getting started', 'course introduction')]

In [110]:
get_prereq_flow_members("css", 2, dict())

{'course introduction': [],
 'css': ['html', 'introduction', 'getting started'],
 'getting started': ['introduction',
  'course introduction',
  'welcome',
  'introduction to the course',
  'intro'],
 'html': ['introduction', 'getting started', 'preparation'],
 'intro': [],
 'introduction': [],
 'introduction to the course': [],
 'preparation': [],
 'welcome': []}

In [50]:
get_cutted_prereq("variables", 1) 

['introduction', 'course introduction', 'setup', 'getting started', 'welcome']